# T Cell CRISPRoff: Day 6 vs Day 35 — DMR Analysis & Methylation Profile Comparison

This notebook compares CRISPRoff-silenced T cells at two time points:
- **Sample A (Day 6):** Early CRISPRoff silencing
- **Sample B (Day 35):** Late/stable CRISPRoff silencing

**Analyses performed:**
1. Load & inspect bedMethyl pileup files
2. Run `modkit dmr pair` to identify differentially methylated CpGs between day 6 and day 35
3. Visualize DMR results (effect sizes, p-values, per-site methylation)
4. Load read-level CG methylation matrices (NPY) for both time points
5. Full methylation profile comparison: heatmaps, per-CpG fractions, distributions
6. Statistical summary across both replicas of day 35

**Region of interest:** `chr1:206583354-206589854` (CD55 locus, 137 CpGs)

---
## 0. Imports & Setup

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from scipy import stats

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'notebook'

from IPython.display import display, HTML

# Add utils to path
utils_path = "/home/michalula/code/epiCausality/epiCode/utils/"
if utils_path not in sys.path:
    sys.path.append(utils_path)

from funcs_dmr_modkit_analyze import (
    load_pileup_bed,
    plot_pileup_df,
    load_dmr_and_parse,
    plot_dmr_summary,
)
from funcs_analize_forward_reverse_extracted_mC_reads import (
    load_padded_reads,
    generate_dataframe,
    generate_CGs_all,
    visualize_CGs_all,
    get_fractions,
)
from funcs_extract_mC_profiles_from_BAMs import get_reference_sequence

date_today = datetime.today().strftime('%Y%m%d')
print(f"Date: {date_today}")
print("Imports OK")

---
## 1. Configuration — Paths & Parameters

In [ ]:
# ── Reference genome ────────────────────────────────────────────────────────
REF_GENOME = "/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
THREADS = 32

# ── Region of interest ──────────────────────────────────────────────────────
REGION_CHR   = "chr1"
REGION_START = 206583354
REGION_END   = 206589854
REGION_STR   = f"{REGION_CHR}:{REGION_START}-{REGION_END}"

# ── Base directories ────────────────────────────────────────────────────────
BASE = "/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells"

# ── Day 6 CRISPRoff ─────────────────────────────────────────────────────────
D6_BASE = f"{BASE}/day_6/croff/analyze_single_reads/dimelo_v2_output"

PILEUP_D6_BED    = f"{D6_BASE}/new_pileup/20251110_filtered_mC07_pileup_CROFF_Day6_Tcells.bed"
PILEUP_D6_BED_GZ = PILEUP_D6_BED + ".gz"

CG_NPY_D6_FOLDER = D6_BASE
CG_NPY_D6_NAME   = (
    "CG_137_padded_reads_day6_CRoff_Tcells_mC0.7_T2Tv2_NoFullyUnmeth_"
    "ovrlap0.9_mismat0.7_mapQ60_modBaseQ10_mCthresh0.7_t2t_v2_0_"
    "chr1:206583354-206589854_2025-09-29_units_combined_numFWD802_numRVS1480.npy"
)

# ── Day 35 CRISPRoff — Replica 1 (primary) ──────────────────────────────────
D35_R1_BASE = f"{BASE}/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output"

PILEUP_D35_R1_BED    = f"{D35_R1_BASE}/new_dmr_pileup/20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed"
PILEUP_D35_R1_BED_GZ = PILEUP_D35_R1_BED + ".gz"

CG_NPY_D35_R1_FOLDER = D35_R1_BASE
CG_NPY_D35_R1_NAME   = (
    "CG_137_padded_reads_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.7_T2Tv2_0_"
    "filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_mCthresh0.7_T2Tv2_0_"
    "chr1:206583354-206589854_2025-11-09_units_combined_numFWD104_numRVS222.npy"
)

# ── Day 35 CRISPRoff — Replica 2b (secondary) ───────────────────────────────
D35_R2_BASE = f"{BASE}/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output"

PILEUP_D35_R2_BED    = f"{D35_R2_BASE}/new_dmr_pileup/20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed"
PILEUP_D35_R2_BED_GZ = PILEUP_D35_R2_BED + ".gz"

CG_NPY_D35_R2_FOLDER = D35_R2_BASE
CG_NPY_D35_R2_NAME   = (
    "CG_137_padded_reads_replica2B_Day35_CRISPRoff_Tcells_postEP_R9minion_threshold_mC0.7_T2Tv2_0_"
    "filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_mCthresh0.7_T2Tv2_0_"
    "chr1:206583354-206589854_2025-11-10_units_combined_numFWD78_numRVS167.npy"
)

# ── Output directory ────────────────────────────────────────────────────────
OUT_DIR = f"{BASE}/compare_conditions/CRoff_day6_vs_day35"
os.makedirs(OUT_DIR, exist_ok=True)

print(f"Output directory: {OUT_DIR}")
print(f"Region: {REGION_STR}")
print()
print("Day 6 pileup  :", Path(PILEUP_D6_BED_GZ).exists())
print("Day 35 R1 pileup:", Path(PILEUP_D35_R1_BED_GZ).exists())
print("Day 35 R2 pileup:", Path(PILEUP_D35_R2_BED_GZ).exists())
print()
print("Day 6 CG NPY  :", Path(CG_NPY_D6_FOLDER, CG_NPY_D6_NAME).exists())
print("Day 35 R1 NPY :", Path(CG_NPY_D35_R1_FOLDER, CG_NPY_D35_R1_NAME).exists())
print("Day 35 R2 NPY :", Path(CG_NPY_D35_R2_FOLDER, CG_NPY_D35_R2_NAME).exists())

---
## 2. Compress & Index Pileup Files

modkit DMR requires bgzip-compressed and tabix-indexed bedMethyl files.  
Run only for files that are not yet compressed.

In [ ]:
%%bash

# Day 6 CRISPRoff pileup
PILEUP_D6="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/croff/analyze_single_reads/dimelo_v2_output/new_pileup/20251110_filtered_mC07_pileup_CROFF_Day6_Tcells.bed"

if [ ! -f "${PILEUP_D6}.gz" ]; then
    echo "Compressing Day 6 pileup..."
    bgzip -k "${PILEUP_D6}"
    tabix -p bed "${PILEUP_D6}.gz"
    echo "Done: Day 6"
else
    echo "Day 6 already compressed: ${PILEUP_D6}.gz"
fi

# Day 35 Replica 1 pileup
PILEUP_D35_R1="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed"

if [ ! -f "${PILEUP_D35_R1}.gz" ]; then
    echo "Compressing Day 35 R1 pileup..."
    bgzip -k "${PILEUP_D35_R1}"
    tabix -p bed "${PILEUP_D35_R1}.gz"
    echo "Done: Day 35 R1"
else
    echo "Day 35 R1 already compressed: ${PILEUP_D35_R1}.gz"
fi

# Day 35 Replica 2b pileup
PILEUP_D35_R2="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed"

if [ ! -f "${PILEUP_D35_R2}.gz" ]; then
    echo "Compressing Day 35 R2b pileup..."
    bgzip -k "${PILEUP_D35_R2}"
    tabix -p bed "${PILEUP_D35_R2}.gz"
    echo "Done: Day 35 R2b"
else
    echo "Day 35 R2b already compressed: ${PILEUP_D35_R2}.gz"
fi

---
## 3. Load & Inspect Pileup Files

In [ ]:
# Load bedMethyl files
df_d6   = load_pileup_bed(PILEUP_D6_BED_GZ)
df_d35r1 = load_pileup_bed(PILEUP_D35_R1_BED_GZ)
df_d35r2 = load_pileup_bed(PILEUP_D35_R2_BED_GZ)

print("=== Day 6 CRISPRoff ===")
print(f"  Sites: {len(df_d6):,}")
print(f"  Median coverage: {df_d6['Nvalid_cov'].median():.0f}")
print(f"  Mean % methylated: {df_d6['percent_modified'].mean():.1f}%")
print()
print("=== Day 35 CRISPRoff — Replica 1 ===")
print(f"  Sites: {len(df_d35r1):,}")
print(f"  Median coverage: {df_d35r1['Nvalid_cov'].median():.0f}")
print(f"  Mean % methylated: {df_d35r1['percent_modified'].mean():.1f}%")
print()
print("=== Day 35 CRISPRoff — Replica 2b ===")
print(f"  Sites: {len(df_d35r2):,}")
print(f"  Median coverage: {df_d35r2['Nvalid_cov'].median():.0f}")
print(f"  Mean % methylated: {df_d35r2['percent_modified'].mean():.1f}%")

In [ ]:
# Filter to region of interest for local inspection
def filter_to_roi(df, chrom, start, end):
    return df[
        (df['chrom'] == chrom) &
        (df['start'] >= start) &
        (df['end']   <= end)
    ].copy().reset_index(drop=True)

roi_d6    = filter_to_roi(df_d6,    REGION_CHR, REGION_START, REGION_END)
roi_d35r1 = filter_to_roi(df_d35r1, REGION_CHR, REGION_START, REGION_END)
roi_d35r2 = filter_to_roi(df_d35r2, REGION_CHR, REGION_START, REGION_END)

print(f"ROI CpG sites — Day 6: {len(roi_d6)},  Day 35 R1: {len(roi_d35r1)},  Day 35 R2b: {len(roi_d35r2)}")
display(roi_d6.head(5))

### 3.1 Pileup Visualizations

In [ ]:
# Use the built-in pileup plotting function (saves interactive HTML to OUT_DIR)
print("Plotting Day 6 pileup...")
plot_pileup_df(roi_d6, OUT_DIR)

print("Plotting Day 35 R1 pileup...")
plot_pileup_df(roi_d35r1, OUT_DIR)

In [ ]:
# Side-by-side % methylation per CpG position across the ROI
fig = go.Figure()

for df_roi, label, color in [
    (roi_d6,    'Day 6 CRoff',      '#e63946'),
    (roi_d35r1, 'Day 35 CRoff R1',  '#457b9d'),
    (roi_d35r2, 'Day 35 CRoff R2b', '#2a9d8f'),
]:
    # Separate strands
    for strand, dash in [('+', 'solid'), ('-', 'dot')]:
        sub = df_roi[df_roi['strand'] == strand]
        fig.add_trace(go.Scatter(
            x=sub['start'],
            y=sub['percent_modified'],
            mode='markers+lines',
            name=f"{label} ({strand})",
            line=dict(color=color, dash=dash),
            marker=dict(size=4),
            opacity=0.85,
        ))

fig.update_layout(
    title="CpG Methylation Across ROI — CRISPRoff Day 6 vs Day 35",
    xaxis_title=f"Genomic position ({REGION_CHR})",
    yaxis_title="% Methylated",
    yaxis_range=[0, 105],
    legend_title="Sample",
    template='plotly_white',
    width=1100, height=500,
)
fig.write_html(f"{OUT_DIR}/{date_today}_pileup_overlay_day6_vs_day35.html")
fig.show()

In [ ]:
# Coverage comparison per position
fig_cov = go.Figure()
for df_roi, label, color in [
    (roi_d6,    'Day 6 CRoff',      '#e63946'),
    (roi_d35r1, 'Day 35 CRoff R1',  '#457b9d'),
    (roi_d35r2, 'Day 35 CRoff R2b', '#2a9d8f'),
]:
    sub = df_roi[df_roi['strand'] == '+']
    fig_cov.add_trace(go.Bar(
        x=sub['start'], y=sub['Nvalid_cov'],
        name=label, marker_color=color, opacity=0.7,
    ))

fig_cov.update_layout(
    title="CpG Coverage Across ROI (forward strand)",
    xaxis_title=f"Genomic position ({REGION_CHR})",
    yaxis_title="Read coverage (N valid)",
    barmode='overlay',
    template='plotly_white',
    width=1100, height=400,
)
fig_cov.write_html(f"{OUT_DIR}/{date_today}_coverage_day6_vs_day35.html")
fig_cov.show()

---
## 4. Run modkit DMR Pair Analysis

Compares Day 6 (sample A) vs Day 35 Replica 1 (sample B) at single-base resolution.  
Positive effect size → higher methylation in Day 6 relative to Day 35.

In [ ]:
%%bash

REF="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
THREADS=32
OUT_DIR="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/compare_conditions/CRoff_day6_vs_day35"

PILEUP_D6="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/croff/analyze_single_reads/dimelo_v2_output/new_pileup/20251110_filtered_mC07_pileup_CROFF_Day6_Tcells.bed.gz"
PILEUP_D35_R1="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl1_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz"

DMR_OUT="${OUT_DIR}/$(date +%Y%m%d)_single_base_mC07_CRoff_day6_vs_day35_R1.bed"

echo "Running modkit dmr pair: Day 6 vs Day 35 R1"
echo "  Sample A (Day 6):    ${PILEUP_D6}"
echo "  Sample B (Day 35 R1): ${PILEUP_D35_R1}"
echo "  Output: ${DMR_OUT}"

modkit dmr pair \
    -a ${PILEUP_D6} \
    -b ${PILEUP_D35_R1} \
    -o ${DMR_OUT} \
    --ref ${REF} \
    --base C \
    --threads ${THREADS} \
    --log-filepath "${OUT_DIR}/dmr_day6_vs_day35_R1.log"

echo "Done. Lines: $(wc -l < ${DMR_OUT})"

In [ ]:
%%bash
# Also run Day 6 vs Day 35 Replica 2b for cross-validation

REF="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
THREADS=32
OUT_DIR="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/compare_conditions/CRoff_day6_vs_day35"

PILEUP_D6="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/croff/analyze_single_reads/dimelo_v2_output/new_pileup/20251110_filtered_mC07_pileup_CROFF_Day6_Tcells.bed.gz"
PILEUP_D35_R2="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_2b/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251110_repl2b_filtered_mC07_pileup_CROFF_Day35_Tcells.bed.gz"

DMR_OUT="${OUT_DIR}/$(date +%Y%m%d)_single_base_mC07_CRoff_day6_vs_day35_R2b.bed"

echo "Running modkit dmr pair: Day 6 vs Day 35 R2b"
modkit dmr pair \
    -a ${PILEUP_D6} \
    -b ${PILEUP_D35_R2} \
    -o ${DMR_OUT} \
    --ref ${REF} \
    --base C \
    --threads ${THREADS} \
    --log-filepath "${OUT_DIR}/dmr_day6_vs_day35_R2b.log"

echo "Done. Lines: $(wc -l < ${DMR_OUT})"

---
## 5. Load & Parse DMR Results

In [ ]:
import glob

# Auto-detect the most recent DMR output files
dmr_files_r1  = sorted(glob.glob(f"{OUT_DIR}/*day6_vs_day35_R1.bed"))
dmr_files_r2b = sorted(glob.glob(f"{OUT_DIR}/*day6_vs_day35_R2b.bed"))

assert dmr_files_r1,  f"No DMR R1 output found in {OUT_DIR}"
assert dmr_files_r2b, f"No DMR R2b output found in {OUT_DIR}"

DMR_PATH_R1  = dmr_files_r1[-1]
DMR_PATH_R2B = dmr_files_r2b[-1]

print(f"DMR R1 file : {DMR_PATH_R1}")
print(f"DMR R2b file: {DMR_PATH_R2B}")

In [ ]:
def parse_modkit_dmr(path):
    """
    Parse modkit dmr pair single-base output (11-column format).
    Columns: chrom, start, end, name, score (log-LR),
             m:n_mod_a, n_total_a, m:n_mod_b (or '.'), n_total_b,
             m:pct_a, m:pct_b (or '.')
    effect_size = Δ% methylation (Day6 − Day35).
    score       = modkit log-likelihood ratio (significance metric).
    """
    def parse_mod(s):
        if str(s) == '.': return np.nan
        try:   return float(str(s).split(':')[1])
        except: return np.nan

    df = pd.read_csv(path, sep='\t', header=None, comment='#')
    df.columns = ['chrom','start','end','name','score',
                  '_mod_count_a','n_total_a','_mod_count_b','n_total_b',
                  '_pct_a_raw','_pct_b_raw']
    for c in ['start','end','n_total_a','n_total_b','score']:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    df['n_mod_a']     = df['_mod_count_a'].apply(parse_mod)
    df['n_mod_b']     = df['_mod_count_b'].apply(parse_mod)
    df['pct_a']       = df['_pct_a_raw'].apply(parse_mod)   # % methylated Day6
    df['pct_b']       = df['_pct_b_raw'].apply(parse_mod)   # % methylated Day35
    df['frac_a']      = df['n_mod_a'] / df['n_total_a']     # fraction modified A
    df['frac_b']      = df['n_mod_b'] / df['n_total_b']     # fraction modified B
    df['effect_size'] = df['pct_a'] - df['pct_b']           # Δ% (positive = more meth Day6)
    df['strand']      = '.'
    return df.drop(columns=['_mod_count_a','_mod_count_b','_pct_a_raw','_pct_b_raw'])

dmr_r1  = parse_modkit_dmr(DMR_PATH_R1)
dmr_r2b = parse_modkit_dmr(DMR_PATH_R2B)

print("DMR R1 shape:", dmr_r1.shape)
print("Columns:", dmr_r1.columns.tolist())
display(dmr_r1.head())

In [ ]:
# Filter to ROI and compute summary statistics
def filter_dmr_to_roi(df, chrom, start, end):
    return df[
        (df['chrom'] == chrom) &
        (df['start'] >= start) &
        (df['end']   <= end)
    ].copy().reset_index(drop=True)

dmr_roi_r1  = filter_dmr_to_roi(dmr_r1,  REGION_CHR, REGION_START, REGION_END)
dmr_roi_r2b = filter_dmr_to_roi(dmr_r2b, REGION_CHR, REGION_START, REGION_END)

print(f"ROI CpGs — DMR R1: {len(dmr_roi_r1)},  DMR R2b: {len(dmr_roi_r2b)}")

for label, df in [('R1', dmr_roi_r1), ('R2b', dmr_roi_r2b)]:
    print(f"\n--- Day 35 {label} ---")
    print(f"  Effect size (Δ%): mean={df['effect_size'].mean():.2f},  std={df['effect_size'].std():.2f}")
    print(f"  Sites effect>0 (more meth Day6):  {(df['effect_size']>0).sum()}")
    print(f"  Sites effect<0 (more meth Day35): {(df['effect_size']<0).sum()}")
    # Use log-likelihood score as significance (|score| > 1 is a common threshold)
    sig = df[df['score'].abs() > 1]
    print(f"  Significant sites (|log-LR score| > 1): {len(sig)}")

---
## 6. DMR Visualizations

In [ ]:
# Note: plot_dmr_summary expects the old 19-column format, so we skip it here
# and use our custom plots below instead.
print("Skipping built-in plot_dmr_summary (incompatible column format).")
print("Using custom plots in the cells below.")

In [ ]:
# Effect size across the ROI — both replicas
fig = make_subplots(rows=2, cols=1,
    shared_xaxes=True,
    subplot_titles=['Day 6 vs Day 35 R1 — Effect Size per CpG',
                    'Day 6 vs Day 35 R2b — Effect Size per CpG'],
    vertical_spacing=0.08)

for row, (df, label) in enumerate([(dmr_roi_r1, 'R1'), (dmr_roi_r2b, 'R2b')], start=1):
    colors = ['#e63946' if v > 0 else '#457b9d' for v in df['effect_size']]
    fig.add_trace(go.Bar(
        x=df['start'], y=df['effect_size'],
        name=f'Day 35 {label}', marker_color=colors,
        showlegend=False,
    ), row=row, col=1)
    fig.add_hline(y=0, line_dash='dash', line_color='black', row=row, col=1)

fig.update_yaxes(title_text='Effect size (Day6 − Day35)', row=1, col=1)
fig.update_yaxes(title_text='Effect size (Day6 − Day35)', row=2, col=1)
fig.update_xaxes(title_text=f'Genomic position ({REGION_CHR})', row=2, col=1)
fig.update_layout(
    title='CRISPRoff Day 6 vs Day 35 — Per-CpG Effect Size<br><sup>Red = more methylated at Day 6 | Blue = more methylated at Day 35</sup>',
    template='plotly_white', width=1100, height=600,
)
fig.write_html(f"{OUT_DIR}/{date_today}_effect_size_day6_vs_day35.html")
fig.show()

In [ ]:
# Per-CpG % methylation scatter: Day 6 vs Day 35
fig = make_subplots(rows=1, cols=2,
    subplot_titles=['Day 6 vs Day 35 R1', 'Day 6 vs Day 35 R2b'])

for col, (df, label) in enumerate([(dmr_roi_r1, 'R1'), (dmr_roi_r2b, 'R2b')], start=1):
    fig.add_trace(go.Scatter(
        x=df['pct_a'],
        y=df['pct_b'],
        mode='markers',
        marker=dict(
            size=6,
            color=df['effect_size'],
            colorscale='RdBu_r',
            cmin=-50, cmax=50,
            colorbar=dict(title='Δ% meth<br>(Day6−Day35)') if col == 2 else None,
            showscale=(col == 2),
        ),
        text=[f"pos: {s}<br>n_a={na:.0f}  n_b={nb:.0f}" for s, na, nb
              in zip(df['start'], df['n_total_a'], df['n_total_b'])],
        name=f'Day 35 {label}',
    ), row=1, col=col)
    # Diagonal
    fig.add_trace(go.Scatter(
        x=[0, 100], y=[0, 100],
        mode='lines', line=dict(dash='dash', color='grey'),
        showlegend=False,
    ), row=1, col=col)

fig.update_xaxes(title_text='% Methylated — Day 6 CRoff', range=[-2, 102])
fig.update_yaxes(title_text='% Methylated — Day 35 CRoff', range=[-2, 102])
fig.update_layout(
    title='Per-CpG Methylation: Day 6 vs Day 35<br><sup>Color = Δ% methylation (Day6 − Day35)</sup>',
    template='plotly_white', width=1100, height=480,
)
fig.write_html(f"{OUT_DIR}/{date_today}_scatter_day6_vs_day35.html")
fig.show()

In [ ]:
# Distribution of effect sizes
fig = go.Figure()
for df, label, color in [(dmr_roi_r1, 'R1', '#457b9d'), (dmr_roi_r2b, 'R2b', '#2a9d8f')]:
    fig.add_trace(go.Histogram(
        x=df['effect_size'], nbinsx=40,
        name=f'Day 35 {label}', marker_color=color, opacity=0.65,
    ))
fig.add_vline(x=0, line_dash='dash', line_color='black')
fig.update_layout(
    title='Distribution of Effect Sizes (Day 6 − Day 35)',
    xaxis_title='Effect size', yaxis_title='Count',
    barmode='overlay', template='plotly_white', width=700, height=400,
)
fig.write_html(f"{OUT_DIR}/{date_today}_effect_size_histogram.html")
fig.show()

In [ ]:
# Reproducibility: effect size R1 vs R2b
merged = dmr_roi_r1[['start', 'effect_size', 'score']].merge(
    dmr_roi_r2b[['start', 'effect_size', 'score']],
    on='start', suffixes=('_R1', '_R2b')
)

r, p = stats.pearsonr(merged['effect_size_R1'].dropna(), merged['effect_size_R2b'].dropna())
print(f"Pearson r (effect size R1 vs R2b): {r:.4f}  (p={p:.2e})")

fig = px.scatter(
    merged, x='effect_size_R1', y='effect_size_R2b',
    labels={'effect_size_R1': 'Δ% meth Day6 vs Day35 R1',
            'effect_size_R2b': 'Δ% meth Day6 vs Day35 R2b'},
    title=f'Replica reproducibility — Δ% methylation<br>Pearson r = {r:.3f}  (p={p:.2e})',
    template='plotly_white',
)
fig.add_trace(go.Scatter(x=[-100,100], y=[-100,100], mode='lines',
    line=dict(dash='dash', color='grey'), showlegend=False))
fig.update_layout(width=600, height=500)
fig.write_html(f"{OUT_DIR}/{date_today}_replica_reproducibility.html")
fig.show()

In [ ]:
# Top differentially methylated CpGs — sorted by |Δ% methylation|
dmr_roi_r1_sorted = dmr_roi_r1.assign(
    abs_effect=dmr_roi_r1['effect_size'].abs()
).sort_values('abs_effect', ascending=False)

print("Top 20 most differentially methylated CpGs (R1):")
cols_show = ['chrom','start','end','n_total_a','pct_a','n_total_b','pct_b','effect_size','score']
display(dmr_roi_r1_sorted[cols_show].head(20))

---
## 7. Load Read-Level Methylation Profiles (NPY)

Load the pre-computed CG methylation matrices for per-read, per-CpG analysis.

In [ ]:
# Load padded CG matrices
CGs_d6    = load_padded_reads(CG_NPY_D6_FOLDER,    CG_NPY_D6_NAME)
CGs_d35r1 = load_padded_reads(CG_NPY_D35_R1_FOLDER, CG_NPY_D35_R1_NAME)
CGs_d35r2 = load_padded_reads(CG_NPY_D35_R2_FOLDER, CG_NPY_D35_R2_NAME)

print(f"Day 6 CRoff    — shape: {CGs_d6.shape}  (reads × CpGs)")
print(f"Day 35 CRoff R1 — shape: {CGs_d35r1.shape}")
print(f"Day 35 CRoff R2b— shape: {CGs_d35r2.shape}")

In [ ]:
# Load reference sequence and full padded reads (region-length arrays)
# These are needed for generate_dataframe / generate_CGs_all.
# The CG_137 matrices (loaded above) are already CpG-extracted; they
# are used directly for heatmaps and per-read fraction analysis.

ref_seq = get_reference_sequence(Path(REF_GENOME), REGION_CHR, REGION_START, REGION_END)
print(f"Reference sequence length: {len(ref_seq)}  (expected: {REGION_END - REGION_START})")

PADDED_D6_NAME   = (
    "padded_reads_day6_CRoff_Tcells_mC0.7_T2Tv2_NoFullyUnmeth_"
    "ovrlap0.9_mismat0.7_mapQ60_modBaseQ10_mCthresh0.7_t2t_v2_0_"
    "chr1:206583354-206589854_2025-09-29.npy"
)
PADDED_D35_R1_NAME = (
    "padded_reads_Tcells_CRISPRoff_Day35_postEP_R9minion_threshold_mC0.7_T2Tv2_0_"
    "filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_mCthresh0.7_T2Tv2_0_"
    "chr1:206583354-206589854_2025-11-09.npy"
)
PADDED_D35_R2_NAME = (
    "padded_reads_replica2B_Day35_CRISPRoff_Tcells_postEP_R9minion_threshold_mC0.7_T2Tv2_0_"
    "filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_mCthresh0.7_T2Tv2_0_"
    "chr1:206583354-206589854_2025-11-10.npy"
)

padded_d6    = load_padded_reads(CG_NPY_D6_FOLDER,    PADDED_D6_NAME)
padded_d35r1 = load_padded_reads(CG_NPY_D35_R1_FOLDER, PADDED_D35_R1_NAME)
padded_d35r2 = load_padded_reads(CG_NPY_D35_R2_FOLDER, PADDED_D35_R2_NAME)

print(f"Padded reads shape — Day 6:    {padded_d6.shape}  (reads × region_length)")
print(f"Padded reads shape — Day 35 R1: {padded_d35r1.shape}")
print(f"Padded reads shape — Day 35 R2b:{padded_d35r2.shape}")

In [ ]:
# Build DataFrames from the full padded reads (region_length columns)
# generate_dataframe labels each column with the reference base at that position
df_d6_reads    = generate_dataframe(padded_d6,    ref_seq)
df_d35r1_reads = generate_dataframe(padded_d35r1, ref_seq)
df_d35r2_reads = generate_dataframe(padded_d35r2, ref_seq)

print(f"Day 6 DataFrame:    {df_d6_reads.shape}")
print(f"Day 35 R1 DataFrame: {df_d35r1_reads.shape}")
print(f"Day 35 R2b DataFrame:{df_d35r2_reads.shape}")
display(df_d6_reads.iloc[:3, :10])

---
## 8. Full Methylation Profile Comparison

### 8.1 Per-Read Heatmaps

In [ ]:
def plot_methylation_heatmap(CG_matrix, title, ax, max_reads=300, cmap='Blues'):
    """
    Plot a read × CpG methylation heatmap.
    Rows = reads, Columns = CpG sites.
    NaN = not covered, 1 = methylated, 0 = unmethylated.
    """
    mat = CG_matrix[:max_reads]
    # Replace NaN with -0.1 for visualization (shown as grey)
    mat_plot = np.where(np.isnan(mat), -0.1, mat)
    cmap_ext = plt.cm.get_cmap(cmap).copy()
    cmap_ext.set_under(color='lightgrey')
    im = ax.imshow(mat_plot, aspect='auto', vmin=0, vmax=1, cmap=cmap_ext,
                   interpolation='nearest')
    ax.set_title(title, fontsize=11)
    ax.set_xlabel('CpG index')
    ax.set_ylabel('Read')
    return im

fig, axes = plt.subplots(1, 3, figsize=(18, 8))

datasets = [
    (CGs_d6,    'Day 6 CRISPRoff\n(mC≥0.7)', 'Blues'),
    (CGs_d35r1, 'Day 35 CRISPRoff R1\n(mC≥0.7)', 'Oranges'),
    (CGs_d35r2, 'Day 35 CRISPRoff R2b\n(mC≥0.7)', 'Greens'),
]

for ax, (mat, title, cmap) in zip(axes, datasets):
    im = plot_methylation_heatmap(mat, title, ax, max_reads=300, cmap=cmap)
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04, label='Methylated')

plt.suptitle('Per-Read CpG Methylation — CRISPRoff T Cells', fontsize=13, y=1.01)
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/{date_today}_heatmap_per_read_day6_vs_day35.png",
            dpi=150, bbox_inches='tight')
plt.show()

### 8.2 Generate CG Analysis (Forward / Reverse strands)

In [ ]:
# Generate CG all matrices (forward and reverse strand CpGs)
CGs_all_d6, C_fwd_d6, G_rev_d6, CG_idx_d6, *_ = generate_CGs_all(
    df_d6_reads, ref_seq, REGION_CHR, REGION_START
)
CGs_all_d35r1, C_fwd_d35r1, G_rev_d35r1, CG_idx_d35r1, *_ = generate_CGs_all(
    df_d35r1_reads, ref_seq, REGION_CHR, REGION_START
)
CGs_all_d35r2, C_fwd_d35r2, G_rev_d35r2, CG_idx_d35r2, *_ = generate_CGs_all(
    df_d35r2_reads, ref_seq, REGION_CHR, REGION_START
)

print(f"CG units — Day 6: {CGs_all_d6.shape[1]}")
print(f"CG units — Day 35 R1: {CGs_all_d35r1.shape[1]}")
print(f"CG units — Day 35 R2b: {CGs_all_d35r2.shape[1]}")

In [ ]:
# Compute per-CpG methylation fractions
frac_d6    = get_fractions(CGs_all_d6)
frac_d35r1 = get_fractions(CGs_all_d35r1)
frac_d35r2 = get_fractions(CGs_all_d35r2)

print(f"Day 6 mean mC fraction:     {np.nanmean(frac_d6):.3f}")
print(f"Day 35 R1 mean mC fraction: {np.nanmean(frac_d35r1):.3f}")
print(f"Day 35 R2b mean mC fraction:{np.nanmean(frac_d35r2):.3f}")

### 8.3 Per-CpG Methylation Fraction Comparison

In [ ]:
# Trim fractions to the same length (number of CG units detected)
n_cgs = min(len(frac_d6), len(frac_d35r1), len(frac_d35r2))
cg_indices = np.arange(1, n_cgs + 1)

fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(cg_indices, frac_d6[:n_cgs],    'o-', color='#e63946', label='Day 6 CRoff',       lw=1.5, ms=4)
ax.plot(cg_indices, frac_d35r1[:n_cgs], 's-', color='#457b9d', label='Day 35 CRoff R1',   lw=1.5, ms=4)
ax.plot(cg_indices, frac_d35r2[:n_cgs], '^-', color='#2a9d8f', label='Day 35 CRoff R2b',  lw=1.5, ms=4, alpha=0.7)

ax.set_xlabel('CpG unit index', fontsize=12)
ax.set_ylabel('Methylation fraction', fontsize=12)
ax.set_title('Per-CpG Methylation Fraction — CRISPRoff Day 6 vs Day 35', fontsize=13)
ax.legend(fontsize=11)
ax.set_ylim(-0.02, 1.02)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{OUT_DIR}/{date_today}_per_CpG_fraction_day6_vs_day35.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Interactive version with Plotly
fig = go.Figure()

for fracs, label, color, symbol in [
    (frac_d6[:n_cgs],    'Day 6 CRoff',     '#e63946', 'circle'),
    (frac_d35r1[:n_cgs], 'Day 35 CRoff R1', '#457b9d', 'square'),
    (frac_d35r2[:n_cgs], 'Day 35 CRoff R2b','#2a9d8f', 'triangle-up'),
]:
    fig.add_trace(go.Scatter(
        x=cg_indices, y=fracs,
        mode='markers+lines',
        name=label,
        line=dict(color=color),
        marker=dict(symbol=symbol, size=6, color=color),
    ))

fig.update_layout(
    title='Per-CpG Methylation Fraction — CRISPRoff Day 6 vs Day 35',
    xaxis_title='CpG unit index',
    yaxis_title='Methylation fraction',
    yaxis_range=[-0.02, 1.02],
    template='plotly_white',
    width=1100, height=450,
)
fig.write_html(f"{OUT_DIR}/{date_today}_per_CpG_fraction_interactive.html")
fig.show()

### 8.4 Clustermap — Read-Level Methylation Patterns

In [ ]:
def plot_clustermap(CGs_all, title, out_path, max_reads=200):
    """
    Hierarchically clustered heatmap of reads × CpGs.
    Uses only rows/columns with sufficient non-NaN data.
    """
    mat = CGs_all.iloc[:max_reads].copy().fillna(-0.1)
    # Only keep CpG columns with at least 20% data
    valid_cols = (CGs_all.notna().mean() >= 0.2)
    mat = mat.loc[:, valid_cols]

    g = sns.clustermap(
        mat,
        cmap='Blues',
        vmin=0, vmax=1,
        figsize=(14, 8),
        yticklabels=False,
        xticklabels=5,
        col_cluster=False,   # Keep CpG order (genomic)
        row_cluster=True,    # Cluster reads by methylation pattern
        cbar_kws={'label': 'Methylated'},
    )
    g.fig.suptitle(title, y=1.01, fontsize=12)
    g.ax_heatmap.set_xlabel('CpG unit index')
    g.ax_heatmap.set_ylabel('Read (clustered)')
    plt.savefig(out_path, dpi=150, bbox_inches='tight')
    plt.show()
    print(f"Saved: {out_path}")

print("Clustermap — Day 6 CRISPRoff")
plot_clustermap(CGs_all_d6,
    'Day 6 CRISPRoff — Read-Level Methylation (clustered)',
    f"{OUT_DIR}/{date_today}_clustermap_day6.png"
)

In [ ]:
print("Clustermap — Day 35 CRISPRoff R1")
plot_clustermap(CGs_all_d35r1,
    'Day 35 CRISPRoff R1 — Read-Level Methylation (clustered)',
    f"{OUT_DIR}/{date_today}_clustermap_day35_R1.png"
)

In [ ]:
print("Clustermap — Day 35 CRISPRoff R2b")
plot_clustermap(CGs_all_d35r2,
    'Day 35 CRISPRoff R2b — Read-Level Methylation (clustered)',
    f"{OUT_DIR}/{date_today}_clustermap_day35_R2b.png"
)

### 8.5 Distribution of Per-Read Methylation Sums

In [ ]:
# Sum of methylated CpGs per read (ignoring NaNs)
sums_d6    = np.nansum(CGs_d6,    axis=1)
sums_d35r1 = np.nansum(CGs_d35r1, axis=1)
sums_d35r2 = np.nansum(CGs_d35r2, axis=1)

# Number of covered CpGs per read (normalization denominator)
cov_d6    = np.sum(~np.isnan(CGs_d6),    axis=1)
cov_d35r1 = np.sum(~np.isnan(CGs_d35r1), axis=1)
cov_d35r2 = np.sum(~np.isnan(CGs_d35r2), axis=1)

# Methylation fraction per read
mfrac_d6    = sums_d6    / np.maximum(cov_d6,    1)
mfrac_d35r1 = sums_d35r1 / np.maximum(cov_d35r1, 1)
mfrac_d35r2 = sums_d35r2 / np.maximum(cov_d35r2, 1)

print("Per-read methylation fraction:")
print(f"  Day 6    — mean: {mfrac_d6.mean():.3f},  median: {np.median(mfrac_d6):.3f},  n={len(mfrac_d6)}")
print(f"  Day 35 R1— mean: {mfrac_d35r1.mean():.3f},  median: {np.median(mfrac_d35r1):.3f},  n={len(mfrac_d35r1)}")
print(f"  Day 35 R2b— mean:{mfrac_d35r2.mean():.3f},  median: {np.median(mfrac_d35r2):.3f},  n={len(mfrac_d35r2)}")

In [ ]:
# Violin + strip plot of per-read methylation fraction
df_mfrac = pd.DataFrame({
    'methylation_fraction': np.concatenate([mfrac_d6, mfrac_d35r1, mfrac_d35r2]),
    'sample': (
        ['Day 6 CRoff'] * len(mfrac_d6) +
        ['Day 35 CRoff R1'] * len(mfrac_d35r1) +
        ['Day 35 CRoff R2b'] * len(mfrac_d35r2)
    ),
})

fig, ax = plt.subplots(figsize=(8, 6))
palette = {'Day 6 CRoff': '#e63946', 'Day 35 CRoff R1': '#457b9d', 'Day 35 CRoff R2b': '#2a9d8f'}

sns.violinplot(data=df_mfrac, x='sample', y='methylation_fraction',
               palette=palette, inner='box', ax=ax, cut=0)
sns.stripplot(data=df_mfrac, x='sample', y='methylation_fraction',
              palette=palette, size=2.5, alpha=0.4, jitter=True, ax=ax)

ax.set_title('Per-Read Methylation Fraction Distribution', fontsize=13)
ax.set_xlabel('Sample')
ax.set_ylabel('Methylation fraction (mC CpGs / covered CpGs)')
ax.set_ylim(-0.02, 1.05)
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/{date_today}_violin_mfrac_day6_vs_day35.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Overlaid KDE of per-read methylation fractions
fig, ax = plt.subplots(figsize=(8, 5))

for fracs, label, color in [
    (mfrac_d6,    'Day 6 CRoff',      '#e63946'),
    (mfrac_d35r1, 'Day 35 CRoff R1',  '#457b9d'),
    (mfrac_d35r2, 'Day 35 CRoff R2b', '#2a9d8f'),
]:
    xs = np.linspace(0, 1, 300)
    if fracs.std() < 1e-8:
        # No variance — plot a spike at the constant value
        ax.axvline(fracs[0], color=color, lw=2, linestyle='--',
                   label=f"{label} (n={len(fracs)}, constant={fracs[0]:.3f})")
    else:
        # Add tiny jitter to avoid singular covariance with near-constant data
        jitter = np.random.default_rng(0).normal(0, 1e-6, size=len(fracs))
        kde = stats.gaussian_kde(fracs + jitter, bw_method=0.15)
        ax.plot(xs, kde(xs), color=color, lw=2, label=f"{label} (n={len(fracs)})")
        ax.fill_between(xs, kde(xs), alpha=0.15, color=color)

ax.set_xlabel('Methylation fraction per read')
ax.set_ylabel('Density')
ax.set_title('KDE — Per-Read Methylation Distribution\nCRISPRoff Day 6 vs Day 35', fontsize=12)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/{date_today}_KDE_mfrac_day6_vs_day35.png", dpi=150, bbox_inches='tight')
plt.show()

---
## 9. Statistical Tests

In [ ]:
# Mann-Whitney U test on per-read methylation fractions
from scipy.stats import mannwhitneyu, ks_2samp

comparisons = [
    ('Day 6 vs Day 35 R1',  mfrac_d6, mfrac_d35r1),
    ('Day 6 vs Day 35 R2b', mfrac_d6, mfrac_d35r2),
    ('Day 35 R1 vs R2b',    mfrac_d35r1, mfrac_d35r2),
]

print("=" * 70)
print(f"{'Comparison':<30} {'Mann-Whitney U':>16} {'p-value':>12} {'KS stat':>10} {'KS p':>10}")
print("=" * 70)

for label, a, b in comparisons:
    mw_u, mw_p = mannwhitneyu(a, b, alternative='two-sided')
    ks_s, ks_p = ks_2samp(a, b)
    sig = '***' if mw_p < 0.001 else ('**' if mw_p < 0.01 else ('*' if mw_p < 0.05 else 'ns'))
    print(f"{label:<30} {mw_u:>16.0f} {mw_p:>12.2e} {ks_s:>10.4f} {ks_p:>10.2e}  {sig}")

print("=" * 70)
print("Significance: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")

In [ ]:
# Per-CpG t-test: Day 6 vs Day 35 (using read-level binary values)
n_cgs = CGs_d6.shape[1]
n_cgs_r1 = CGs_d35r1.shape[1]
n_cgs_cmp = min(n_cgs, n_cgs_r1)

pvals   = np.full(n_cgs_cmp, np.nan)
effects = np.full(n_cgs_cmp, np.nan)

for i in range(n_cgs_cmp):
    a = CGs_d6[:, i]
    b = CGs_d35r1[:, i]
    a = a[~np.isnan(a)]
    b = b[~np.isnan(b)]
    if len(a) >= 5 and len(b) >= 5:
        _, pvals[i] = stats.ttest_ind(a, b, equal_var=False)
        effects[i] = a.mean() - b.mean()

sig_cpgs = np.sum(pvals < 0.05)
print(f"Per-CpG t-test (Day 6 vs Day 35 R1):")
print(f"  Total CpGs tested: {n_cgs_cmp}")
print(f"  Significant (p<0.05): {sig_cpgs} ({100*sig_cpgs/n_cgs_cmp:.1f}%)")
print(f"  Higher meth in Day 6: {np.sum(effects[pvals<0.05]>0)} | Higher meth in Day 35: {np.sum(effects[pvals<0.05]<0)}")

In [ ]:
# Volcano-style plot: effect size vs -log10(p-value)
fig, ax = plt.subplots(figsize=(8, 6))

valid = ~np.isnan(pvals)
log_p = -np.log10(pvals[valid])
eff   = effects[valid]

colors_pt = np.where((pvals[valid] < 0.05) & (eff > 0),  '#e63946',   # Sig, higher Day 6
            np.where((pvals[valid] < 0.05) & (eff < 0),  '#457b9d',   # Sig, higher Day 35
            'lightgrey'))                                               # Not significant

ax.scatter(eff, log_p, c=colors_pt, s=20, alpha=0.8, edgecolors='none')
ax.axhline(-np.log10(0.05), color='black', linestyle='--', lw=1, label='p=0.05')
ax.axvline(0, color='black', linestyle='--', lw=1)

ax.set_xlabel('Effect size (Day 6 − Day 35 methylation fraction)', fontsize=11)
ax.set_ylabel('-log₁₀(p-value)', fontsize=11)
ax.set_title('Per-CpG Differential Methylation\nCRISPRoff Day 6 vs Day 35 R1', fontsize=12)

from matplotlib.patches import Patch
ax.legend(handles=[
    Patch(color='#e63946', label=f'Higher Day 6 (p<0.05)'),
    Patch(color='#457b9d', label=f'Higher Day 35 (p<0.05)'),
    Patch(color='lightgrey', label='Not significant'),
], fontsize=9)

plt.tight_layout()
plt.savefig(f"{OUT_DIR}/{date_today}_volcano_per_CpG_day6_vs_day35.png", dpi=150, bbox_inches='tight')
plt.show()

---
## 10. Summary & Export

In [ ]:
# Build a summary DataFrame combining pileup + per-CpG DMR statistics
summary = roi_d6[['chrom', 'start', 'end', 'strand',
                  'Nvalid_cov', 'percent_modified']].copy()
summary = summary.rename(columns={
    'Nvalid_cov': 'cov_d6',
    'percent_modified': 'pct_meth_d6',
})

# Merge Day 35 R1 pileup
d35r1_sub = roi_d35r1[['start', 'strand', 'Nvalid_cov', 'percent_modified']].rename(
    columns={'Nvalid_cov': 'cov_d35r1', 'percent_modified': 'pct_meth_d35r1'}
)
summary = summary.merge(d35r1_sub, on=['start', 'strand'], how='outer')

# Merge Day 35 R2b pileup
d35r2_sub = roi_d35r2[['start', 'strand', 'Nvalid_cov', 'percent_modified']].rename(
    columns={'Nvalid_cov': 'cov_d35r2', 'percent_modified': 'pct_meth_d35r2'}
)
summary = summary.merge(d35r2_sub, on=['start', 'strand'], how='outer')

# Merge DMR stats (R1) — use effect_size (Δ%) and score (log-LR)
dmr_sub = dmr_roi_r1[['start', 'effect_size', 'score', 'n_total_a', 'n_total_b']].rename(
    columns={'effect_size': 'delta_pct_R1', 'score': 'logLR_score_R1',
             'n_total_a': 'dmr_n_d6', 'n_total_b': 'dmr_n_d35r1'}
)
summary = summary.merge(dmr_sub, on='start', how='left')

# Delta from pileup (independent check)
summary['delta_pct_pileup_R1']  = summary['pct_meth_d6'] - summary['pct_meth_d35r1']
summary['delta_pct_pileup_R2b'] = summary['pct_meth_d6'] - summary['pct_meth_d35r2']

summary_path = f"{OUT_DIR}/{date_today}_summary_CRoff_day6_vs_day35.csv"
summary.to_csv(summary_path, index=False)

print(f"Summary saved: {summary_path}")
display(summary.head(10))

In [ ]:
# Final statistics summary
print("=" * 60)
print("  ANALYSIS SUMMARY: CRISPRoff Day 6 vs Day 35")
print("=" * 60)
print(f"  Region: {REGION_STR}")
print()
print("  Pileup statistics (ROI):")
print(f"    Day 6    — {len(roi_d6)} CpGs,  mean cov={roi_d6['Nvalid_cov'].mean():.0f},  mean meth={roi_d6['percent_modified'].mean():.1f}%")
print(f"    Day 35 R1— {len(roi_d35r1)} CpGs,  mean cov={roi_d35r1['Nvalid_cov'].mean():.0f},  mean meth={roi_d35r1['percent_modified'].mean():.1f}%")
print(f"    Day 35 R2b— {len(roi_d35r2)} CpGs,  mean cov={roi_d35r2['Nvalid_cov'].mean():.0f},  mean meth={roi_d35r2['percent_modified'].mean():.1f}%")
print()
print("  Per-read methylation fraction:")
print(f"    Day 6    — mean={mfrac_d6.mean():.3f}, n={len(mfrac_d6)} reads")
print(f"    Day 35 R1— mean={mfrac_d35r1.mean():.3f}, n={len(mfrac_d35r1)} reads")
print(f"    Day 35 R2b— mean={mfrac_d35r2.mean():.3f}, n={len(mfrac_d35r2)} reads")
print()
print("  DMR results — Day 6 vs Day 35 R1 (ROI):")
print(f"    CpGs in ROI: {len(dmr_roi_r1)}")
print(f"    Mean Δ% methylation (Day6−Day35): {dmr_roi_r1['effect_size'].mean():.2f}%")
print(f"    More meth Day 6 (Δ>0):  {(dmr_roi_r1['effect_size']>0).sum()} CpGs")
print(f"    More meth Day 35 (Δ<0): {(dmr_roi_r1['effect_size']<0).sum()} CpGs")
print(f"    |logLR score| > 1:      {(dmr_roi_r1['score'].abs()>1).sum()} CpGs")
print()
print("  Files saved to:")
import glob
for f in sorted(glob.glob(f"{OUT_DIR}/{date_today}_*")):
    print(f"    {Path(f).name}")
print("=" * 60)